In [2]:
### This will look for all the autoTMA images, and check if the corresponding cores are on the server
### if not, generate and upload them
## It also generate a composite core of the H&E and the TMA core
import dsaSecrets, io
import girder_client, os
%load_ext autoreload
%autoreload 2
import tmaHelpers as tma

gc = girder_client.GirderClient(apiUrl=dsaSecrets.apiUrl)
gc.authenticate(apiKey=dsaSecrets.apiToken)
## Samples Images 62bb03e4e70def4ae72689f5

{'_id': '5cacfa86e62914004a60e139'}

In [3]:
tmaCoreInfo = {}  ### I'm going to stuck all of the information for all of the TMA slides into this dictionary for processing
### This will grab all the ['B7']
studyImageId = '62ded6cbe70def4ae726b228'
studyImages = list(gc.listItem(studyImageId))
for s in studyImages:
    autoTMAdata = tma.getAutoTMAcoreInfo(gc,s['_id'])
    tmaCoreInfo[s['name']] = {'itemInfo': s, 'autoTMAdata': autoTMAdata}
## This gets the individual slides that have the tmas on them

In [5]:
### get information for the TMA cores that are on the server..
tmaCorePath = '/collection/RING Ki67/Cores'
tcp = gc.get('resource/lookup?path=%s' % tmaCorePath)

tmaCoreImages = list(gc.listItem(tcp['_id']))
print(len(tmaCoreImages),'TMA Cores have been uploaded')

uploadedTmaCores = [x['name'] for x in tmaCoreImages]

997 TMA Cores have been uploaded


In [6]:
roiNameSet = []
for i in studyImages:
    print("Processing",i['name'])
    roiNameSet.extend( tma.export_tma_cores(gc,i['_id'],uploadedTmaCores,roiDir='/mnt/c/devel/tmaCores') )

Processing 254461.svs
Processing 254462.svs
Processing 257591.svs
Processing 257593.svs
Processing 257595.svs
Processing 257597.svs
Processing 257600.svs
Processing 257601.svs
Processing 257603.svs
Processing 257604.svs


In [7]:
## Delete cores that were uploaded but no longer match current centroid i.e. I updated the base image
for x in tmaCoreImages:
    if x['name'] not in roiNameSet:
        print(x['name'],x['_id'])
        gc.delete('item/%s' % x['_id'])

S11.G14.45735.24473.tiff 62e9d9b9e70def4ae727d44a
S11.G15.48839.24019.tiff 62e9d9bae70def4ae727d44d
S11.H15.48925.27538.tiff 62e9d9c2e70def4ae727d47a
S11.I1.5568.32119.tiff 62e9d9c3e70def4ae727d47d
S11.I10.33495.31437.tiff 62e9d9c8e70def4ae727d498
S11.I11.36598.31361.tiff 62e9d9c8e70def4ae727d49b
S11.I15.49011.31057.tiff 62e9d9cbe70def4ae727d4a7
S11.I2.8671.32043.tiff 62e9d9c3e70def4ae727d480
S11.I3.11774.31967.tiff 62e9d9c4e70def4ae727d483
S11.I4.14877.31892.tiff 62e9d9c4e70def4ae727d486
S11.I5.17980.31816.tiff 62e9d9c5e70def4ae727d489
S11.I6.21083.31740.tiff 62e9d9c6e70def4ae727d48c
S11.I7.24186.31664.tiff 62e9d9c6e70def4ae727d48f
S11.I8.27289.31588.tiff 62e9d9c7e70def4ae727d492
S11.I9.30392.31512.tiff 62e9d9c7e70def4ae727d495
S11.J10.33581.34956.tiff 62e9d9d0e70def4ae727d4c5
S11.J12.39787.34804.tiff 62e9d9d1e70def4ae727d4cb
S11.J13.42890.34728.tiff 62e9d9d2e70def4ae727d4ce
S11.J15.49009.34707.tiff 62e9d9d3e70def4ae727d4d4
S11.J8.27375.35107.tiff 62e9d9cfe70def4ae727d4bf
S11.J9.30478

In [10]:
### Generate the pairs of images for the multicore yaml file..
## An HE image should be paired with the corresponding 
### The H&E images are 17 and 18 
### These need to be mapped
import random
import tqdm

clearYamlFolder = False
random.seed(42)

HEslideDict = {}
### get information for the TMA cores that are on the server..
tmaCorePath = '/collection/RING Ki67/Cores'
tcp = gc.get('resource/lookup?path=%s' % tmaCorePath)

tmaCoreImages = list(gc.listItem(tcp['_id']))
print(len(tmaCoreImages),'TMA Cores have been uploaded')



for c in tmaCoreImages:
    (slide,coreId,cx,cy,ext) = c['name'].split(".")
    if slide in ['S17','S18']:
        if coreId not in HEslideDict:
            HEslideDict[coreId] = c
        else:
            print("Duplicate HE core detected for",coreId,slide)

## Now that I have the HEs to pair, I Can loop through all the cores and generate the composite

yamlRootFolder = '62e9287be70def4ae7275e94'

## GEt a list of all the existing YAML files in the target directory


if clearYamlFolder:
    #curYamlFileList = gc.listResource(f'resource/{yamlRootFolder}/items?type=folder')
    curYamlFileList = gc.get(f'resource/{yamlRootFolder}/items?type=folder&limit=5000')
    for y in tqdm.tqdm(curYamlFileList):
        gc.delete('item/%s' % y['_id'])
    curlYamlFileList = []
else:
    curYamlFileList = [x['name'] for x in gc.get(f'resource/{yamlRootFolder}/items?type=folder&limit=5000')]  ## NEED TO MAKE THIS RECURSIVE IN NEXT ITERATION
    

print(len(curYamlFileList),'yaml files already generated')
    
batchFolderList = list(gc.listFolder(yamlRootFolder))
random.shuffle(tmaCoreImages) ## Shuffle core images


for i,c in tqdm.tqdm( enumerate(tmaCoreImages)):
    (slide,coreId,cx,cy,ext) = c['name'].split(".")
    if slide  not in ['S17','S18']:
        ## So these are then.. NOT H&E cores
        yamlFileName = f'{slide}.{coreId}.yaml'
        if yamlFileName not in curYamlFileList:
            batchFolderId = batchFolderList[i% len(batchFolderList)]['_id']
#             try:
            mcYaml = tma.writeMultiCoreYaml(gc, c['_id'], HEslideDict[coreId]['_id'] )
            f = io.StringIO(mcYaml)
            gc.uploadFile(batchFolderId,f,yamlFileName,len(mcYaml),parentType='folder')
#             except:
#                 print("Some error with",slide,coreId)

975 TMA Cores have been uploaded
336 yaml files already generated


975it [03:04,  5.29it/s]  


In [ ]:
HEslideDict.keys()

In [ ]:
c

In [ ]:
# def getAutoTMAcoreInfo( gc, imgId, tmaAnnotationName='autoTMA'):
#     ### This will find and return the latest autoTMA record associated with a given image
#     i = gc.get('item/%s' % imgId)
#     fileRoot = i['meta']['slideNumber']
    
#     ## Get autoTMA documents for this item
#     tmaDocs=  gc.get('annotation?itemId=%s&name=%s' % (imgId, tmaAnnotationName))
#     print(len(tmaDocs),'autoTMA annotations found... will use the most recent one')
    
#     if tmaDocs:
#         ae = gc.get('annotation/%s' % tmaDocs[-1]['_id']) ## Grab the last one... assume that's the most recent.. may need to check
#         print(len(ae['annotation']['elements']),'total ROIs to export')
#     else:
#         ae = None
#         print("No tma cores found for this image..")
    
#     return ae
#     roiDir = 'ROI_Export'
    
#     if not os.path.isdir(roiDir):
#         os.mkdir(roiDir)

#     for roi in ae['annotation']['elements']:
#         roiFileName = "%s/%s.%s.%d.%d.tiff" % (roiDir,fileRoot,roi['label']['value'],roi['center'][0],roi['center'][1])
        
#         if not os.path.isfile(roiFileName):
#             print(roiFileName)
#             r= tma.get_annotation_region(gc,imgId,roi)        

#             with open(roiFileName,"wb") as fpi:
#                 fpi.write(r)
        
       